In [ ]:
import os, os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import re
import cv2

In [2]:
directory = 'MC2-Image-Data/'

In [3]:
file_list = []
caption_list = []
text_list = []
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.csv'):
            file_list.append(os.path.abspath(root)+ '/' + file)
        if 'caption.txt' in file:
            caption_list.append(os.path.abspath(root)+ '/' + file)
        if 'text' in file:
            text_list.append(os.path.abspath(root) + '/' + file)
# caption_list

In [4]:
# text_list

In [5]:
def read_csv(path):
    df = pd.read_csv(path, encoding= 'unicode_escape',dtype=str)
    df['PersonId'] = os.path.split(path)[1][6:-4]
    return df

def read_txt(path, colName):
    df = pd.read_csv(path, sep='\t', encoding= 'unicode_escape', header=None, names=[colName])
    name = os.path.split(path)[1]
    if 'text' in name:
        df['TextId'] = name[6:-4]
    else:
        df['PersonId'] = os.path.split(path)[1][6:-11]
    return df


cap_list = []
txt_list = []
for file in caption_list:
    cap_df = read_txt(file, 'Caption')
    cap_list.append(cap_df)
for file in text_list:
    txt_list.append(read_txt(file, 'Text'))
cap = pd.concat(cap_list).reset_index(drop=True)
txt = pd.concat(txt_list).reset_index(drop=True)

In [6]:
cap.head()

,Caption,PersonId
0,Tortoise racing for life,33_42
1,Be the pop of color that can illuminate the mo...,33_17
2,Tortoise at Gym building Muscles,33_31
3,Rolling dice in a stable position,33_53
4,The things are not always where you think they...,33_1


In [10]:
txt.head()

,Text,TextId
0,You come home to find a single lit pink candle...,35_text3
1,Green turtles are a good investment Buy now!,35_text2
2,This trophy is a good prize for my win in HotS,35_text1
3,The benefit of the maple leaf pencil is that y...,35_text5
4,My cats enjoyed playing soccer with this eyeba...,35_text4


In [11]:
cap.to_json('caption.json', orient='records')
txt.to_json('text.json', orient='records')

In [12]:
df_list = []
for file in file_list:
    df = read_csv(file)
    df_list.append(df)


In [13]:
concat_df = pd.concat(df_list).reset_index(drop=True)
concat_df.shape

(4491, 7)

In [14]:
concat_df.head(10)

,x,y,Width,Height,Score,Label,PersonId
0,377,202,1237,1502,0.31465,cloudSign,33_40
1,2003,515,296,1361,0.48349,pinkCandle,33_40
2,229,273,2137,1825,0.34838,pumpkinNotes,33_40
3,1560,111,1210,2127,0.36762,sign,33_40
4,515,565,1341,2608,0.29639,birdCall,33_54
5,666,1143,1482,2514,0.29539,pumpkinNotes,33_54
6,797,1748,1200,1398,0.31314,redWhistle,33_54
7,11,269,1089,3750,0.47006,sign,33_54
8,1422,2366,1250,1680,0.3209,silverStraw,33_54
9,595,1143,1170,861,0.27061,blueSunglasses,33_54


In [15]:
columns = ['x','y','Width','Height']
for col in columns:
    try:
        concat_df[col].astype('str').astype('int')
    except:
        pass

In [16]:
concat_df.to_json('data.json', orient='records')

In [157]:
concat_df.dtypes

x           object
y           object
Width       object
Height      object
Score       object
Label       object
PersonId    object
dtype: object

In [75]:
df = pd.merge(concat_df, concat_txt, how='outer', on='PersonId')
df[df['Label'].isnull()]

,x,y,Width,Height,Score,Label,PersonId,Caption
4506,NaN,NaN,NaN,NaN,NaN,NaN,27_13,I found a cute pumpkin notebook next to sticky...


In [77]:
df[df['x'].isnull()]

,x,y,Width,Height,Score,Label,PersonId,Caption
4506,NaN,NaN,NaN,NaN,NaN,NaN,27_13,I found a cute pumpkin notebook next to sticky...


In [78]:
df

,x,y,Width,Height,Score,Label,PersonId,Caption
0,377,202,1237,1502,0.31465,cloudSign,33_40,Rocket in colorful Multiverse
1,2003,515,296,1361,0.48349,pinkCandle,33_40,Rocket in colorful Multiverse
2,229,273,2137,1825,0.34838,pumpkinNotes,33_40,Rocket in colorful Multiverse
3,1560,111,1210,2127,0.36762,sign,33_40,Rocket in colorful Multiverse
4,515,565,1341,2608,0.29639,birdCall,33_54,Tortoise rising high after climbing the tower
...,...,...,...,...,...,...,...,...
4502,476,224,328,1382,0.61677,sign,15_3,hello world written on the sign
4503,4,6,600,934,0.30253,noisemaker,15_1,supporting the lads with these noisemaker sticks
4504,636,763,540,843,0.3176,pumpkinNotes,15_1,supporting the lads with these noisemaker sticks
4505,900,6,304,955,0.34451,sign,15_1,supporting the lads with these noisemaker sticks


In [155]:
df.to_json('data_with_cap.json', orient='records')

In [164]:
d = concat_df.groupby(['PersonId'],as_index=False).agg({'Score': np.max })
d

,PersonId,Score
0,10_1,0.46997
1,10_10,0.40247
2,10_11,0.3996
3,10_12,0.51042
4,10_13,0.42838
...,...,...
889,9_5,0.65481
890,9_6,0.40167
891,9_7,0.59368
892,9_8,0.60256


In [ ]:
concat_df.groupby()

In [168]:
concat_df.groupby(['Label'],as_index=False).agg({'Score': np.max })

,Label,Score
0,birdCall,0.66718
1,blueSunglasses,0.61751
2,canadaPencil,0.57062
3,cloudSign,0.72959
4,cupcakePaper,0.60502
5,eyeball,0.81999
6,hairClip,0.89167
7,lavenderDie,0.91488
8,metalKey,0.66809
9,noisemaker,0.83048


In [173]:
concat_df['Label'].value_counts()

eyeball           501
cloudSign         482
sign              465
pinkCandle        415
blueSunglasses    352
birdCall          324
yellowBag         286
pumpkinNotes      271
hairClip          231
lavenderDie       200
redWhistle        181
cupcakePaper      135
metalKey          134
canadaPencil      116
stickerBox         92
partyFavor         91
trophy             68
silverStraw        58
yellowBalloon      30
noisemaker         26
paperPlate         22
vancouverCards     11
Name: Label, dtype: int64

In [196]:
person2 = concat_df[concat_df['PersonId'].str.startswith('2_')]
person2

,x,y,Width,Height,Score,Label,PersonId
1583,928,14,1381,2218,0.31342,birdCall,2_11
1584,1150,108,1795,3441,0.3148,canadaPencil,2_11
1585,1129,404,928,1707,0.32516,cloudSign,2_11
1586,1492,242,1119,1519,0.28149,metalKey,2_11
1587,1069,27,1069,2030,0.28713,partyFavor,2_11
...,...,...,...,...,...,...,...
1676,474,2124,988,1210,0.43177,metalKey,2_8
1677,2349,538,263,1519,0.33862,pinkCandle,2_8
1678,918,1465,545,2581,0.26701,sign,2_8
1679,1543,1116,988,941,0.26741,blueSunglasses,2_8


In [7]:
im = cv2.imread('MC2-Image-Data/Person1/Person1_1.jpg')

In [ ]:
im.shape